# Append Data
Reads a csv into a dataframe, chunks the dataframe, publishes the first chunk, then appends the remaining chunks. This showcases the **[`edit_features`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features)** function to append data when working with a [Spatially Enabled DataFrame](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/). This method sends the JSON data. In ArcGIS Online, there's also an **[`append`](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.append)** method that upserts data from a previously existing file item (which we'd [love](https://github.com/mpayson/esri-partner-tools/issues/13) a sample for!)

In [3]:
import pandas as pd
from arcgis.gis import GIS

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`cool_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/cool_utils.py) and [`feature_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/feature_utils.py)

In [4]:
from partnerutils.cool_utils import chunk_df
from partnerutils.feature_utils import sdf_from_xyz

### Input

In [5]:
# this will prompt for a password
# can also do GIS("https://www.arcgis.com", "<USERNAME>", "<PASSWORD>")
gis = GIS(username="mpayson_startups")

Enter password:  ········


In [7]:
# path to data
csv_path = "../sample_data/NYC_Restaurant_Inspections.csv"

x_col = "Longitude"
y_col = "Latitude"

### Execution

In [8]:
# read csv and construct spatial dataframe
df = pd.read_csv(csv_path)
sdf = sdf_from_xyz(df, x_col, y_col)
len(sdf)

5000

In [10]:
# iterate through chunks to create and append data
lyr = None
for c_df in chunk_df(sdf):
    if not lyr:
        item = c_df.spatial.to_featurelayer("MyFeatureService", tags="esri_partner_tools_sample")
        lyr = item.layers[0]
    else:
        # THIS IS THE APPEND DATA PART
        fs = c_df.spatial.to_featureset()
        success = lyr.edit_features(adds=fs)
item

<Item title:"MyFeatureService" type:Feature Layer Collection owner:mpayson_startups>

### Clean up

In [ ]:
delete_items = gis.content.search("tags:esri_partner_tools_sample")
gis.content.delete_items(delete_items)